In [ ]:
import cf_xarray  # noqa: F401
import copernicusmarine
import numpy as np
import pandas as pd
import xarray as xr
from seapopym.standard.units import StandardUnitsLabels

In [ ]:
export_folder = "/Users/ash/Documents/Workspaces/Data/phd/SEAPODYM_LMTL/2025-01-10_-_CAFE_HOT/data"
cafe_path = "../1_1_Forcing/raw/CAFE_NPP_8D_25KM_1998_2022.nc"

HOT station coordinates:


In [ ]:
LATITUDE = 22.75
LONGITUDE = -158
POSITION_TOLERANCE = 1
START_TIME = "2005-01-01"
END_TIME = "2010-01-01"

## CMEMS


In [ ]:
coordinates = {
    "minimum_latitude": LATITUDE - POSITION_TOLERANCE,
    "maximum_latitude": LATITUDE + POSITION_TOLERANCE,
    "minimum_longitude": LONGITUDE - POSITION_TOLERANCE,
    "maximum_longitude": LONGITUDE + POSITION_TOLERANCE,
    "start_datetime": START_TIME,
    "end_datetime": END_TIME,
}

bio = copernicusmarine.open_dataset(dataset_id="cmems_mod_glo_bgc_my_0.083deg-lmtl_PT1D-i", **coordinates)
physic = copernicusmarine.open_dataset(dataset_id="cmems_mod_glo_bgc_my_0.083deg-lmtl-Fphy_PT1D-i", **coordinates)

In [ ]:
bio.load()

In [ ]:
physic.load()

In [ ]:
physic = physic.interp_like(bio)

In [ ]:
with xr.set_options(keep_attrs=True):
    data = xr.merge([bio, physic])
    data["T"].attrs["units"] = str(StandardUnitsLabels.temperature.units)
data

## CAFE


In [ ]:
cafe_data = xr.open_dataset(cafe_path)

cafe_data = cafe_data.rename({"lat": "latitude", "lon": "longitude"})
cafe_data["CAFE"] = cafe_data["CAFE"].assign_attrs({"units": "mg m-2 day-1", "weight": "carbon"})
cafe_data_hot = cafe_data.sel(
    latitude=slice(LATITUDE - 2 * POSITION_TOLERANCE, LATITUDE + 2 * POSITION_TOLERANCE),
    longitude=slice(LONGITUDE - 2 * POSITION_TOLERANCE, LONGITUDE + 2 * POSITION_TOLERANCE),
    time=slice(START_TIME, END_TIME),
)
cafe_data_hot.load()

In [ ]:
cafe_data_hot = cafe_data_hot.interp_like(data)
cafe_data_hot

In [ ]:
data["cafe"] = cafe_data_hot["CAFE"]

## Create mask


In [ ]:
cafe_data_hot.CAFE.isnull().sum("time").plot()

In [ ]:
cafe_data_hot.CAFE.isnull().sum(["latitude", "longitude"]).plot()

In [ ]:
data["mask"] = xr.full_like(data["T"], 2, dtype=np.int64)
mask = data["mask"][0, 0].drop_vars(["time", "depth"])
mask.latitude.attrs = {
    "standard_name": "latitude",
    "long_name": "latitude",
    "units": "degrees_north",
    "axis": "Y",
}
mask.longitude.attrs = {
    "standard_name": "longitude",
    "long_name": "longitude",
    "units": "degrees_east",
    "axis": "X",
}
mask.attrs = {}
mask.to_netcdf(f"{export_folder}/mask.nc")
xr.load_dataset(f"{export_folder}/mask.nc")

## To float 64


In [ ]:
data = data.astype(np.float64)

## Export


In [ ]:
for date in data["T"].time.values:
    date_as_string = pd.to_datetime(str(date)).strftime("%Y%m%d")
    data.sel(time=[date])[["U", "V", "T", "cafe"]].to_netcdf(f"{export_folder}/data_{date_as_string}.nc")

This shell commands can be run from seapodym-lmtl directory.

```bash
❯ mpirun -mca btl ^openib -np 6 ./build/bin/seapodym-lmtl -P -G D1N1 -V error /<path_to>/Data/phd/SEAPODYM_LMTL/2024-12-13_-_Constant_fields_no_transport/zpk.tmpl.xml
```

```bash
❯ mpirun -mca btl ^openib -np 6 ./build/bin/seapodym-lmtl -B -G D1N1 -V error /<path_to>/Data/phd/SEAPODYM_LMTL/2024-12-13_-_Constant_fields_no_transport/zpk.tmpl.xml
```
